In [4]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import array_to_img, img_to_array, load_img

# Define the data generator
data_gen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load all images in the directory
data_dir = 'dataset/train/cercospera'
files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.jpg')]
x_train = np.array([np.array(load_img(f)) for f in files])

# Generate augmented data
data_gen.fit(x_train)
flow = data_gen.flow(x_train, batch_size=32)

# Save the augmented images
new_images_dir = 'new_images'
if not os.path.exists(new_images_dir):
    os.makedirs(new_images_dir)

for i, batch in enumerate(flow):
    for j in range(batch.shape[0]):
        img = array_to_img(batch[j], scale=True)
        img.save(os.path.join(new_images_dir, 'augmented_image_{}_{}.jpg'.format(i, j)))
        if i * 32 + j >= x_train.shape[0]:
            break
    if i * 32 >= x_train.shape[0]:
        break

ImportError: cannot import name 'array_to_img' from 'keras.preprocessing.image' (C:\Users\rafae\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\preprocessing\image.py)

In [7]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import save_img, img_to_array

# Define the data generator
data_gen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load all images in the directory
data_dir = 'dataset/train/cercospera'
files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.jpg')]
x_train = np.array([img_to_array(load_img(f)) for f in files])

# Generate augmented data
data_gen.fit(x_train)
flow = data_gen.flow(x_train, batch_size=32)

# Save the augmented images
new_images_dir = 'new_images'
if not os.path.exists(new_images_dir):
    os.makedirs(new_images_dir)

for i, batch in enumerate(flow):
    for j in range(batch.shape[0]):
        img = batch[j].astype('uint8')
        save_img(os.path.join(new_images_dir, 'augmented_image_{}_{}.jpg'.format(i, j)), img)
        if i * 32 + j >= x_train.shape[0]:
            break
    # if i * 32 >= x_train.shape[0]:
        break


ImportError: cannot import name 'save_img' from 'keras.preprocessing.image' (C:\Users\rafae\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\preprocessing\image.py)

In [6]:
# from random import seed
from tensorflow import keras
import tensorflow as tf

ds = keras.preprocessing.image_dataset_from_directory(
    "dataset/train",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    batch_size=1,
    image_size=(512, 512),
    shuffle=False,
)

class_names = ds.class_names

def augment(image, label):
    seed = tf.random
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.5)
    image = tf.image.random_contrast(image, lower=0.5, upper=1)
    image = tf.image.random_saturation(image, lower=0.5, upper=1)
    image = tf.image.random_hue(image, max_delta=0.1)
    image = tf.image.random_crop(image, size=[1, 384, 384, 3])
    image = tf.image.central_crop(image, central_fraction=0.75)
    image = tf.image.rot90(image, k=1)
    return image, label

ds = ds.map(augment)
# 256 + 128 = 384

def save_image(image, path_dest, name):
    image = image.numpy().astype("uint8")
    image = np.squeeze(image, axis=0)
    image = tf.image.encode_jpeg(image, quality=100, optimize_size=True, chroma_downsampling=True)
    tf.io.write_file(path_dest + "/" + name, image)

i = 0
# for i in range(ds.cardinality().numpy()):
for image, label in ds:
    i += 1
    # image, label = next(iter(ds))
    class_name = class_names[label.numpy().argmax()]
    # print(class_name)
    filename = f"{i}.jpg"
    save_image(image, class_name, filename)

Found 110 files belonging to 2 classes.


NameError: name 'np' is not defined

In [70]:
image, label = next(iter(ds))
label

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[1., 0.]], dtype=float32)>

In [24]:
from tensorflow import keras
import tensorflow as tf
import numpy as np

def augment(image, label):
    image = tf.image.rot90(image, k=1) if tf.random.uniform(()) > 0.5 else image
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=1.0)
    image = tf.image.random_saturation(image, lower=0.75, upper=1.25)
    image = tf.image.random_crop(image, size=[1, 448, 448, 3])
    return image, label


def save_image(image, path_dest, name):
    image = image.numpy().astype("uint8")
    image = np.squeeze(image, axis=0)
    image = tf.image.encode_jpeg(image, quality=100, optimize_size=False, chroma_downsampling=False)
    tf.io.write_file(path_dest + "/" + name, image)


for i in range(7):
    ds = keras.preprocessing.image_dataset_from_directory(
            "images",
            labels="inferred",
            label_mode="categorical",
            class_names=None,
            image_size=(512, 512),
            shuffle=False,
            batch_size=1,
        )

    class_names = ds.class_names

    ds = ds.map(augment)

    for j, (image, label) in enumerate(ds):
        file_index  = (i * 64) + j
        class_name = class_names[label.numpy().argmax()]
        filename = f"{file_index}.jpg"
        save_image(image, class_name, filename)

Found 64 files belonging to 1 classes.
Found 64 files belonging to 1 classes.
Found 64 files belonging to 1 classes.
Found 64 files belonging to 1 classes.
Found 64 files belonging to 1 classes.
Found 64 files belonging to 1 classes.
Found 64 files belonging to 1 classes.


In [8]:
import tensorflow as tf

seed = np.random.randint(0, 1000)

<module 'tensorflow._api.v2.random' from 'C:\\Users\\rafae\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python310\\site-packages\\tensorflow\\_api\\v2\\random\\__init__.py'>